Network for arteficial data

In [20]:
import pandas as pd
import numpy as np
import torch
from scipy.integrate import odeint

In [21]:
data = pd.read_csv('/workspaces/bio-pinn/Arteficial Data/Data_noisy.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
y0 = np.array([1., 0., 0.2, 0.])
print(data)


     time         ConcA     ConcB         ConcC         ConcD
0     0.0  8.383181e+00  0.700421  7.317251e-01  3.763695e+00
1     0.1  7.381639e+00  2.353448 -1.491672e-01  2.799064e+00
2     0.2  9.669862e+00  4.151044 -5.257714e-01  2.079049e+00
3     0.3  1.068910e+01  6.608603 -1.002894e+00  4.513447e-01
4     0.4  1.318064e+01  7.992387 -2.278684e+00 -1.237712e+00
..    ...           ...       ...           ...           ...
96    9.6  3.236100e+19 -8.284169  3.236100e+19 -3.236100e+19
97    9.7  5.335427e+19 -7.540272  5.335427e+19 -5.335427e+19
98    9.8  8.796632e+19 -8.860039  8.796632e+19 -8.796632e+19
99    9.9  1.450319e+20 -7.734428  1.450319e+20 -1.450319e+20
100  10.0  2.391173e+20 -8.072378  2.391173e+20 -2.391173e+20

[101 rows x 5 columns]


In [22]:
class Net(torch.nn.Module):
    
    def __init__(self, k):
        super().__init__()
        self.actf = torch.tanh
        self.f1 = torch.nn.Linear(1, 100)
        self.f2 = torch.nn.Linear(100, 100)
        self.f3 = torch.nn.Linear(100, 100)
        self.f4 = torch.nn.Linear(100, 4)
        
        self.k1 = torch.nn.Parameter(torch.tensor(k[0], device=device))
        self.k2 = torch.nn.Parameter(torch.tensor(k[1], device=device))
        self.k3 = torch.nn.Parameter(torch.tensor(k[2], device=device))
        self.k4 = torch.nn.Parameter(torch.tensor(k[3], device=device))

    def forward(self, x):
        print(f'Input shape: {x.shape}')
        x = self.actf(self.f1(x))
        print(f'After f1: {x.shape}')
        x = self.actf(self.f2(x))
        print(f'After f2: {x.shape}')
        x = self.actf(self.f3(x))
        print(f'After f3: {x.shape}')
        x = self.f4(x)
        print(f'After f4: {x.shape}')
        return x.squeeze()

from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, in_tensor, out_tensor):
        self.inp = in_tensor
        self.out = out_tensor

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.out[idx]

In [23]:
from torch.autograd import grad

def phys_loss(t, C, model):
    dCdt = torch.autograd.grad(
        C, t,
        grad_outputs=torch.ones_like(C),
        create_graph=True
    )[0]

    A, B, Cc, D = C[:,0], C[:,1], C[:,2], C[:,3]

    fA = -model.k1 * A
    fB = model.k1 * A - model.k2 * B
    fC = model.k2 * B - model.k3 * Cc
    fD = model.k3 * Cc - model.k4 * D

    rhs = torch.stack([fA, fB, fC, fD], dim=1)
    return torch.mean((dCdt - rhs)**2)

In [37]:
k = [1., 1., 1., 1.]
from torch.optim import Adam
model_pinn = Net(k).to(device)

epochs = 1000
optimizer_pinn = Adam(model_pinn.parameters(), lr=0.1)
loss_fcn = torch.nn.MSELoss()
df=data[['ConcA','ConcB','ConcC','ConcD']]

train_in = torch.tensor(data['time'].values, dtype=torch.float32, requires_grad=True).view(-1, 1).to(device)
train_out = torch.tensor(df.values, dtype=torch.float32).to(device)
time=torch.tensor(data['time'].values, dtype=torch.float32)


for epoch in range(epochs):
    model_pinn.train()  # Ensure model is in training mode

    optimizer_pinn.zero_grad()
    pred = model_pinn(train_in)
    base_loss = loss_fcn(pred, train_out)
    phys_loss_value = phys_loss(train_in, pred, model_pinn)
    total_loss = base_loss + phys_loss_value
    total_loss.backward()
    optimizer_pinn.step()

    # Print losses for every epoch
    print(f'Epoch {epoch+1}/{epochs} | Loss: {total_loss.item():.4f} = {base_loss.item():.4f} (Base) + {phys_loss_value.item():.4f} (Physics)')






Input shape: torch.Size([101, 1])
After f1: torch.Size([101, 100])
After f2: torch.Size([101, 100])
After f3: torch.Size([101, 100])
After f4: torch.Size([101, 4])
Epoch 1/1000 | Loss: inf = inf (Base) + 0.0058 (Physics)
Input shape: torch.Size([101, 1])
After f1: torch.Size([101, 100])
After f2: torch.Size([101, 100])
After f3: torch.Size([101, 100])
After f4: torch.Size([101, 4])
Epoch 2/1000 | Loss: inf = inf (Base) + 5.7728 (Physics)
Input shape: torch.Size([101, 1])
After f1: torch.Size([101, 100])
After f2: torch.Size([101, 100])
After f3: torch.Size([101, 100])
After f4: torch.Size([101, 4])
Epoch 3/1000 | Loss: inf = inf (Base) + 324.4354 (Physics)
Input shape: torch.Size([101, 1])
After f1: torch.Size([101, 100])
After f2: torch.Size([101, 100])
After f3: torch.Size([101, 100])
After f4: torch.Size([101, 4])
Epoch 4/1000 | Loss: inf = inf (Base) + 428.7269 (Physics)
Input shape: torch.Size([101, 1])
After f1: torch.Size([101, 100])
After f2: torch.Size([101, 100])
After f3: to